In [3]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import math

In [5]:
data = pd.read_json("./cve_list.txt")
data = dict(data)
data

{0: 0      CVE-2017-1000503
 1        CVE-2017-11418
 2        CVE-2017-10887
 3         CVE-2017-9434
 4        CVE-2017-17575
              ...       
 448      CVE-2017-16014
 449      CVE-2017-11439
 450       CVE-2017-2789
 451       CVE-2017-3156
 452      CVE-2017-17280
 Name: 0, Length: 453, dtype: object,
 1: 0      {'entities': [['V2_CVSS', 6.8], ['VULNERABILIT...
 1      {'entities': [['DRIVER', 'dapur/apps/app_artic...
 2      {'entities': [['CWE', 'CWE-426'], ['VULNERABIL...
 3      {'entities': [['SOFTWARE', 'cryptopp'], ['SOUR...
 4      {'entities': [['SOURCECODE', 'vendor_details.p...
                              ...                        
 448    {'entities': [['VULNERABILITY', 'dos'], ['V2_C...
 449    {'entities': [['VENDER', 'sitecore'], ['CWE', ...
 450    {'entities': [['SOFTWARE', 'ichitaro office'],...
 451    {'entities': [['VENDER', 'apache'], ['CWE', 'C...
 452    {'entities': [['VULNERABILITY', 'info'], ['V2_...
 Name: 1, Length: 453, dtype: object}

In [6]:
entities = []

for d in data[1]:
    d['entities'] = dict(d['entities'])
    entities.append(d['entities'])

entities = pd.DataFrame(entities)
entities

,V2_CVSS,VULNERABILITY,STRUCT,CWE,IMPACT,VENDER,DRIVER,SOURCECODE,SOFTWARE,FUNCTION
0,6.8,execcode,race condition,CWE-362,6.4,jenkins,NaN,NaN,NaN,NaN
1,7.5,sql,NaN,CWE-89,6.4,NaN,dapur/apps/app_article/controller/,article_list.php,fiyo cms,NaN
2,9.3,priv,untrusted search path,CWE-426,10.0,NaN,NaN,NaN,book walker,NaN
3,5.0,unknown,NaN,CWE-125,2.9,NaN,NaN,zinflate.c,crypto,NaN
4,7.5,sql,NaN,CWE-89,6.4,NaN,NaN,item_details.php,fs groupon clone,NaN
...,...,...,...,...,...,...,...,...,...,...
448,5.0,dos,NaN,CWE-388,2.9,NaN,NaN,NaN,http-proxy,NaN
449,3.5,xss,NaN,CWE-79,2.9,sitecore,shell/applications/tools/run,NaN,sitecore,NaN
450,7.5,execcode overflow,NaN,CWE-119,6.4,justsystems,NaN,NaN,ichitaro office,NaN
451,5.0,unknown,NaN,CWE-361,2.9,apache,NaN,NaN,cxf,NaN


In [11]:
def Similarity(list1, list2):
    count = 0
    for i in range(len(list1)):
        if type(list1[i]) == type(list2[i]):
            if type(list1[i]) == type('str'):
                if list1[i] == list2[i]:
                    count += 1
            else:
                if not(math.isnan(list1[i]) or math.isnan(list2[i])):
                    if list1[i] == list2[i]:
                        count += 1
    return count/len(list1)

In [15]:
weight = []
idx = 0

for i in range(len(entities)):
    for j in range(i+1, len(entities)):
        s = Similarity(entities.loc[i, :].tolist(), entities.loc[j, :].tolist())
        if(s != 0):
            weight.append([])
            weight[idx].append(i)
            weight[idx].append(j)
            weight[idx].append(s)
            idx += 1

weight = pd.DataFrame(weight)
weight.columns = ['Source', 'Target', 'Weight']
weight

,Source,Target,Weight
0,0,1,0.1
1,0,4,0.1
2,0,7,0.2
3,0,13,0.2
4,0,14,0.1
...,...,...,...
42234,448,451,0.2
42235,448,452,0.1
42236,449,451,0.1
42237,449,452,0.1


In [16]:
weight.to_excel("EdgeTable.xlsx")#, index = False)